In [ ]:
# TODO: come up with better graph 
def visualise(perturbed_df, true_df, org_name, auto_org_name, org_name_query): 
    pd_perturbed_age_sum = perturbed_df.toPandas()
    pd_true_age_sum = true_df.groupBy('SSIC').sum(org_name).sort(asc(auto_org_name)).toPandas()
    true_perturbed_age_sum = pd.merge(pd_perturbed_age_sum, pd_true_age_sum, on='SSIC', how='inner')
    
    print(true_perturbed_age_sum)
    # error = perc_error(results['True'], results['Geom-DP-'+str(epsilon)])
    
    ax = true_perturbed_age_sum.plot.bar(rot=45, figsize=(8, 5), width=0.8, fontsize=12, colormap = 'Accent')
    plt.title('Income Distribution (n = '+str(300)+')', fontsize=16)
    # plt.legend(fontsize = 12, labels=labels)
    # print(true_perturbed_age_sum)
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.5)
    plt.ylabel('Count', fontsize = 14)
    plt.ylim(-50)
    return plt 


# ax = results.plot.bar(rot=45, figsize=(13, 8), width=0.8, fontsize=12, colormap = 'Accent')
#     plt.title('Income Distribution (n = '+str(sample_size)+')', fontsize=16)
#     plt.legend(fontsize = 12, labels=labels)
#     plt.grid(axis='y', alpha=0.5)
#     plt.ylabel('Count', fontsize = 14)
#     plt.ylim(-50)
#     plt.show()
# visualise(dp_private_data, public_data, "innovation_expenditure", "sum(innovation_expenditure)", "innovation_expenditure_sum").show()


In [ ]:
# Laplacian noise
sum_privacy_budget = 10 
CLAMPING_BOUNDS = [0, 5000]

session = Session.from_dataframe(
    privacy_budget=PureDPBudget(float('inf')),
    source_id="sectors_analysis",
    dataframe=public_data
)

# group by query 
edu_sum_age_query = (
    QueryBuilder("sectors_analysis")
    .groupby(sectors)
    .sum("innovation_expenditure", low=CLAMPING_BOUNDS[0], high=CLAMPING_BOUNDS[1]) 
)

# privacy budget for the query
dp_private_data = session.evaluate(
    edu_sum_age_query,
    privacy_budget=PureDPBudget(sum_privacy_budget), 
)

# # show visualisations
visualise(dp_private_data, public_data, "innovation_expenditure", "sum(innovation_expenditure)", "innovation_expenditure_sum").show()
public_data.groupby(public_data.SSIC).count().show()

In [ ]:
e_1 = [] 
no_points = 1000

for i in range(no_points):
    e_1.append(np.random.laplace(loc=0, scale=1/0.1))
    
    
e_2 = []
for i in range(no_points):
    e_2.append(np.random.laplace(loc=0, scale=1/2))
    
    
e_3 = []
for i in range(no_points):
    e_3.append(np.random.laplace(loc=0, scale=1/5))
    
    
e_4 = []
for i in range(no_points):
    e_4.append(np.random.laplace(loc=0, scale=1/10))

In [ ]:
import matplotlib.cm as cm

# x = np.arange(10)
# print(x)
# ys = [i+x+(i*x)**2 for i in range(10)]
# print(len(ys))

e = [e_1, e_2, e_3, e_4]
# print(e)
colors = cm.rainbow(np.linspace(0, 1, len(e)))

x = [0.1, 2, 5, 10]
# print(x[0]*100)
for i, (y, c) in enumerate(zip(e, colors)):
    k = [x[i]]*1000    
    print(x[i], c)
    plt.scatter(k, y, color=c)

In [ ]:
# Laplacian noise
sum_privacy_budget = 0.5  
CLAMPING_BOUNDS_1 = [0, 600]

# session = Session.from_dataframe(
#     privacy_budget=PureDPBudget(float('inf')),
#     source_id="sectors_analysis",
#     dataframe=public_data
# )

# group by query 
count_firms_query = (
    QueryBuilder("sectors_analysis")
    .groupby(sectors)
    .count("UEN")#, low=CLAMPING_BOUNDS_1[0], high=CLAMPING_BOUNDS_1[1]) 
)

# privacy budget for the query
dp_private_data = session.evaluate(
    count_firms_query,
    privacy_budget=PureDPBudget(PRIVACY_BUDGET), 
)

# # show visualisations
visualise(dp_private_data, public_data, "UEN", "count(UEN)", "UEN_count").show()
public_data.groupby(public_data.UEN).count().show()

In [ ]:
session = Session.from_dataframe(
    # adding laplacian noise 
    privacy_budget=RhoZCDPBudget(PRIVACY_BUDGET), # privacy budget for the session
    source_id="members",
    dataframe=sampled_data
)


In [ ]:

# each individual has an influence on at most one bucket (SECTORS)
edu_levels = KeySet.from_dict({
    "education_level": [
        # "up-to-high-school",
        # "high-school-diploma",
        # "bachelors-associate",
        # "masters-degree",
        "doctorate-professional",
    ]
})

session = Session.from_dataframe(
    # adding laplacian noise 
    privacy_budget=RhoZCDPBudget(PRIVACY_BUDGET), # privacy budget for the session
    source_id="members",
    dataframe=sampled_data
)

# group by query 
edu_sum_age_query = (
    QueryBuilder("members")
    .groupby(edu_levels)
    .sum("age", low=CLAMPING_BOUNDS[0], high=CLAMPING_BOUNDS[1]) 
)

# privacy budget for the query
edu_sum_ages = session.evaluate(
    edu_sum_age_query,
    privacy_budget=RhoZCDPBudget(PRIVACY_BUDGET), #RhoZCDPBudget
)

# show visualisations
visualise(edu_sum_ages, sampled_data).show()
sampled_data.groupby(sampled_data.education_level).count().show()

In [ ]:
# out of all intervals computed at the 95% level, 95% of them should contain the parameter's true value
Factors affecting the width of the CI include the confidence level, the sample size, and the variability in the sample

In [ ]:
Yeah that is right, although we currently only support Discrete Gaussian noise (not regular Gaussian noise)
Here are two links that might be helpful

A utility function for calculating noise scales (it is in tmlt.core which is a dependency of tmlt.analytics)
https://docs.tmlt.dev/core/latest/reference/tmlt/core/utils/parameters/index.html#tmlt.core.utils.parameters.calculate_noise_scale


The cmf for the Discrete Gaussian distribution. We don’t currently support an inverse cmf function, which is what you would want to get the interval, but you can get the interval using a snippet like this
https://docs.tmlt.dev/core/latest/reference/tmlt/core/utils/distributions/index.html#tmlt.core.utils.distributions.discrete_gaussian_cmf


In [ ]:
# discrete_gaussian_interval.py
from tmlt.core.utils.distributions import discrete_gaussian_cmf

noise_scale = 7
upper = 0
p = 0.95
while discrete_gaussian_cmf(upper, noise_scale**2) <= 1 - ((1 - p)/2): 
    print(discrete_gaussian_cmf(upper, noise_scale**2), "----", upper)
    # print(discrete_gaussian_cmf(upper, noise_scale**2))
    upper += 1

interval = (-upper, upper)

In [ ]:
while discrete_gaussian_cmf(upper, noise_scale**2) <= 0.975:
    upper += 1

In [ ]:
# print(session._stability["members"])
from scipy.stats import laplace

# formula is noise_scale = stability/epsilon
noise_scale_exact = session._stability["sectors_analysis"]/sum_privacy_budget
noise_scale_float = float(noise_scale_exact)

# https://en.wikipedia.org/wiki/Laplace_distribution
noise_rvs = laplace(loc=0, scale=noise_scale_float) # b value for Laplace distribution

print(noise_rvs)

print(f"sensitivity: {session._stability['sectors_analysis']}")
print(sum_privacy_budget)
print(f"noise scale: {noise_scale_float}")
print(f"variance: {noise_rvs.var()}")

#Endpoints of the range that contains fraction alpha [0, 1] of the distribution
#interval(alpha, loc=0, scale=1)
print(f"confidence interval: {noise_rvs.interval(0.95)}")